#I. Quick Setups

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Aug 21 19:37:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime > "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.


#II. Prepare Data

In [ ]:
%env HOME=/content/drive/MyDrive
%cd ~/Research/ner/intent_classifier/final_result
!ls -lrt

env: HOME=/content/drive/MyDrive
/content/drive/MyDrive/Research/ner/intent_classifier/final_result
total 38716
-rw------- 1 root root 13121130 Aug 31 22:18 kb_eval_final.json
-rw------- 1 root root  8636219 Aug 31 22:19 updated_DSTC9_train_labels.json
-rw------- 1 root root  1191914 Aug 31 22:19 updated_DSTC9_val_labels.json
-rw------- 1 root root   848616 Aug 31 22:19 updated_DSTC9_excluded_eval_labels.json
-rw------- 1 root root    76794 Aug 31 22:19 updated_DSTC10_clean_val_labels.json
-rw------- 1 root root  1883004 Aug 31 22:20 updated_labels_wkb.zip
-rw------- 1 root root    11998 Aug 31 22:22 intent_examples_val.json
-rw------- 1 root root  3020592 Aug 31 22:22 intent_examples_train1.json
-rw------- 1 root root    12071 Aug 31 22:23 intent_examples_val_clean.json
-rw------- 1 root root   672102 Aug 31 22:23 intent_examples.zip
-rw------- 1 root root    33326 Sep  1 00:39 V2_intent_examples_val.json
-rw------- 1 root root  6313393 Sep  1 00:39 V2_intent_examples_train1.json
-rw-

In [ ]:
import json
ff = open('kb_eval_final.json','r')
kb = json.load(ff)
ff.close()
dic = kb

##Data wo 0 cluster

In [ ]:
!zip intent_examples.zip intent_examples*.json

updating: intent_examples_train1.json (deflated 84%)
updating: intent_examples_train2.json (deflated 84%)
updating: intent_examples_val_clean.json (deflated 80%)
updating: intent_examples_val.json (deflated 78%)


In [ ]:
#Val set, specific on DSTC10_val_log
import json

res = []
title_map = {}
pos_count = 0
neg_count = 0

for i in range(len(rlabels_val)):
    curr = rlabels_val[i]
    candidate = {}
    candidate['sentence'] = rlog_val[i][-1]['clean_text']#or 'text'
    if candidate['sentence'] in title_map:
        continue
    else:
        title_map[candidate['sentence']] = 1
    if curr['target'] == 1:
        candidate['label'] = curr['knowledge'][0]['cluster_id']
        pos_count += 1
        res.append(candidate)
    else:
        candidate['label'] = 0
        neg_count += 1
print('Final pos# ', pos_count)
print('Final neg# ', neg_count)

out_val = open('intent_examples_val_clean.json','w')
json.dump(res, out_val, indent = 4)
out_val.close()

Final pos#  103
Final neg#  158


###Exp1: train with kb titles, dstc9 train | val | excluded_eval

In [ ]:
#Train set, 1st experiment, containing DSTC9 train, val, exclude_eval, and kb titles
import json
import random

res = []
title_map = {}#we only want unique
#First kb titles, all positives
for cat in kb:
    for elem in kb[cat]:
        doc_lst = kb[cat][elem]['docs']
        for entry in doc_lst:
            if doc_lst[entry]['title'] in title_map:
                continue
            title_map[doc_lst[entry]['title']] = 1
            candidate = {}
            candidate['sentence'] = doc_lst[entry]['title']
            candidate['label'] = doc_lst[entry]['intent_label']
            res.append(candidate)


pos_count = len(res)#count positives
neg_count = 0
log_lst = [log_train, log_val, log_eval]
label_lst = [labels_train, labels_val, labels_eval]

for x in range(len(log_lst)):
    log = log_lst[x]
    label = label_lst[x]
    for i in range(len(label)):
        curr = label[i]
        candidate = {}
        candidate['sentence'] = log[i][-1]['text']
        if candidate['sentence'] in title_map:
            continue
        else:
            title_map[candidate['sentence']] = 1
        if curr['target'] == 1:
            pos_count += 1
            candidate['label'] = curr['knowledge'][0]['cluster_id']
            res.append(candidate)
print('Final pos# ', pos_count)
print('Final neg# ', neg_count)

random.shuffle(res)
out = open('intent_examples_train1.json', 'w')
json.dump(res, out, indent = 4)
out.close()

Final pos#  28206
Final neg#  0


###Exp2: train with kb titles and eval_excluded

Final pos#  11115

Final neg#  1991

In [ ]:
#Train set, 1st experiment, containing DSTC9 train, val, exclude_eval, and kb titles
import json
import random

res = []
title_map = {}#we only want unique
#First kb titles, all positives
for cat in kb:
    for elem in kb[cat]:
        doc_lst = kb[cat][elem]['docs']
        for entry in doc_lst:
            if doc_lst[entry]['title'] in title_map:
                continue
            title_map[doc_lst[entry]['title']] = 1
            candidate = {}
            candidate['sentence'] = doc_lst[entry]['title']
            candidate['label'] = doc_lst[entry]['intent_label']
            res.append(candidate)


pos_count = len(res)#count positives
neg_count = 0
log_lst = [log_eval]
label_lst = [labels_eval]

for x in range(len(log_lst)):
    log = log_lst[x]
    label = label_lst[x]
    for i in range(len(label)):
        curr = label[i]
        candidate = {}
        candidate['sentence'] = log[i][-1]['text']
        if candidate['sentence'] in title_map:
            continue
        else:
            title_map[candidate['sentence']] = 1
        if curr['target'] == 1:
            pos_count += 1
            candidate['label'] = curr['knowledge'][0]['cluster_id']
            res.append(candidate)
print('Final pos# ', pos_count)
print('Final neg# ', neg_count)

random.shuffle(res)
out = open('intent_examples_train2.json', 'w')
json.dump(res, out, indent = 4)
out.close()

Final pos#  11115
Final neg#  0


##Data with 0 cluster

### Prepare label files

In [ ]:
%cd updated_labels
!ls -lrt

/content/drive/My Drive/Research/ner/intent_classifier/updated_labels
total 20108
-rw------- 1 root root    80772 Aug 12 03:13 updated_DSTC10_track2_task2_labels_phon_templates.json
-rw------- 1 root root  2172285 Aug 12 06:44 updated_DSTC9_task1_val_labels_phon_templates.json
-rw------- 1 root root 15769778 Aug 12 06:44 updated_DSTC9_task1_train_labels_phon_templates.json
-rw------- 1 root root  1586506 Aug 12 06:44 updated_DSTC9_task1_eval_labels_phon_templates.json
-rw------- 1 root root    80772 Aug 12 06:44 updated_DSTC10_clean_val_labels.json
-rw------- 1 root root   899259 Aug 12 06:44 updated_exclude_labels.json


In [ ]:
import json

def _process_labels(infile):
    f1 = open(infile, 'r')
    labels = json.load(f1)
    f1.close()
    
    for label in labels:
        if label['target'] == 1:
            for elem in label['knowledge']:
                cluster_id = dic[elem['domain']][str(elem['entity_id'])]['docs'][str(elem['doc_id'])]['intent_label']
                elem['cluster_id'] = cluster_id
        else:
            label['cluster_id'] = 0
    new_name = infile.split('/')[-1]
    out = open('updated_'+new_name, 'w')
    json.dump(labels, out, indent=4)
    out.close()

In [ ]:
_process_labels('/content/drive/MyDrive/Research/ner/DSTC9_task1_val_labels_phon_templates.json')
_process_labels('/content/drive/MyDrive/Research/ner/DSTC9_task1_train_labels_phon_templates.json')
_process_labels('/content/drive/MyDrive/Research/ner/DSTC9_task1_eval_labels_phon_templates.json')
_process_labels('/content/drive/MyDrive/Research/ner/intent_classifier/DSTC10_clean_val_labels.json')

In [ ]:
_process_labels('../exclude_labels.json')

In [ ]:
!zip updated_labels.zip *.json

  adding: updated_DSTC10_track2_task2_labels_phon_templates.json (deflated 93%)
  adding: updated_DSTC9_task1_eval_labels_phon_templates.json (deflated 91%)
  adding: updated_DSTC9_task1_train_labels_phon_templates.json (deflated 93%)
  adding: updated_DSTC9_task1_val_labels_phon_templates.json (deflated 93%)


###Load Label files

In [ ]:
%cd updated_labels

[Errno 2] No such file or directory: 'updated_labels'
/content/drive/My Drive/Research/ner/intent_classifier/updated_labels


In [ ]:
f1 = open('updated_DSTC9_task1_train_labels_phon_templates.json', 'r')
labels_train = json.load(f1)
f1.close()

f2 = open('updated_DSTC9_task1_val_labels_phon_templates.json', 'r')
labels_val = json.load(f2)
f2.close()

f3 = open('updated_exclude_labels.json', 'r')#if og then it would be cheat
labels_eval = json.load(f3)
f3.close()

f4 = open('updated_DSTC10_clean_val_labels.json', 'r')
rlabels_val = json.load(f4)
f4.close()


In [ ]:
#OR
f1 = open('updated_DSTC9_train_labels.json', 'r')
labels_train = json.load(f1)
f1.close()

f2 = open('updated_DSTC9_val_labels.json', 'r')
labels_val = json.load(f2)
f2.close()

f3 = open('updated_DSTC9_excluded_eval_labels.json', 'r')#if og then it would be cheat
labels_eval = json.load(f3)
f3.close()

f4 = open('updated_DSTC10_clean_val_labels.json', 'r')
rlabels_val = json.load(f4)
f4.close()

In [ ]:
%cd ..

/content/drive/MyDrive/Research/ner/intent_classifier


### Load Log files

In [ ]:
f1 = open('/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data/train/logs.json','r')
log_train = json.load(f1)
f1.close()

f2 = open('/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data/val/logs.json','r')
log_val = json.load(f2)
f2.close()

f3 = open('/content/drive/MyDrive/Research/ner/intent_classifier/exclude_template_logs.json','r')#if og then it would be cheat
log_eval = json.load(f3)
f3.close()

f4 = open('/content/drive/MyDrive/Research/ner/intent_classifier/DSTC10_clean_val_logs.json','r')
rlog_val = json.load(f4)
f4.close()

###Generate Examples



Final pos#  104

Final neg#  159 (for og file)

Final pos#  103

Final neg#  158 (for clean file)


In [ ]:
#Val set, specific on DSTC10_val_log
import json

res = []
title_map = {}
pos_count = 0
neg_count = 0

for i in range(len(rlabels_val)):
    curr = rlabels_val[i]
    candidate = {}
    candidate['sentence'] = rlog_val[i][-1]['text']#or 'clean_text'
    if candidate['sentence'] in title_map:
        continue
    else:
        title_map[candidate['sentence']] = 1
    if curr['target'] == 1:
        candidate['label'] = curr['knowledge'][0]['cluster_id']
        pos_count += 1
    else:
        candidate['label'] = 0
        neg_count += 1
    res.append(candidate)
print('Final pos# ', pos_count)
print('Final neg# ', neg_count)

out_val = open('V2_intent_examples_val.json','w')#Change here as well
json.dump(res, out_val, indent = 4)
out_val.close()

Final pos#  103
Final neg#  158


###Exp1: train with kb titles, dstc9 train | val | excluded_eval

Final pos#  28206

Final neg#  27998

In [ ]:
#Train set, 1st experiment, containing DSTC9 train, val, exclude_eval, and kb titles
import json
import random

res = []
title_map = {}#we only want unique
#First kb titles, all positives
for cat in dic:
    for elem in dic[cat]:
        doc_lst = dic[cat][elem]['docs']
        for entry in doc_lst:
            if doc_lst[entry]['title'] in title_map:
                continue
            title_map[doc_lst[entry]['title']] = 1
            candidate = {}
            candidate['sentence'] = doc_lst[entry]['title']
            candidate['label'] = doc_lst[entry]['intent_label']
            res.append(candidate)


pos_count = len(res)#count positives
neg_count = 0
log_lst = [log_train, log_val, log_eval]
label_lst = [labels_train, labels_val, labels_eval]

for x in range(len(log_lst)):
    log = log_lst[x]
    label = label_lst[x]
    for i in range(len(label)):
        curr = label[i]
        candidate = {}
        candidate['sentence'] = log[i][-1]['text']
        if candidate['sentence'] in title_map:
            continue
        else:
            title_map[candidate['sentence']] = 1
        if curr['target'] == 1:
            pos_count += 1
            candidate['label'] = curr['knowledge'][0]['cluster_id']
            res.append(candidate)
        else:
            if neg_count < pos_count:
                neg_count += 1
                candidate['label'] = 0
                res.append(candidate)
print('Final pos# ', pos_count)
print('Final neg# ', neg_count)

random.shuffle(res)
out = open('V2_intent_examples_train1.json', 'w')
json.dump(res, out, indent = 4)
out.close()

Final pos#  28206
Final neg#  27998


###Exp2: train with kb titles and eval_excluded

Final pos#  11115

Final neg#  1991

In [ ]:
#Train set, 1st experiment, containing DSTC9 train, val, exclude_eval, and kb titles
import json
import random

res = []
title_map = {}#we only want unique
#First kb titles, all positives
for cat in dic:
    for elem in dic[cat]:
        doc_lst = dic[cat][elem]['docs']
        for entry in doc_lst:
            if doc_lst[entry]['title'] in title_map:
                continue
            title_map[doc_lst[entry]['title']] = 1
            candidate = {}
            candidate['sentence'] = doc_lst[entry]['title']
            candidate['label'] = doc_lst[entry]['intent_label']
            res.append(candidate)


pos_count = len(res)#count positives
neg_count = 0
log_lst = [log_eval]
label_lst = [labels_eval]

for x in range(len(log_lst)):
    log = log_lst[x]
    label = label_lst[x]
    for i in range(len(label)):
        curr = label[i]
        candidate = {}
        candidate['sentence'] = log[i][-1]['text']
        if candidate['sentence'] in title_map:
            continue
        else:
            title_map[candidate['sentence']] = 1
        if curr['target'] == 1:
            pos_count += 1
            candidate['label'] = curr['knowledge'][0]['cluster_id']
            res.append(candidate)
        else:
            if neg_count < pos_count:
                neg_count += 1
                candidate['label'] = 0
                res.append(candidate)
print('Final pos# ', pos_count)
print('Final neg# ', neg_count)

random.shuffle(res)
out = open('V2_intent_examples_train2.json', 'w')
json.dump(res, out, indent = 4)
out.close()

Final pos#  11115
Final neg#  1991


##Organizing

In [ ]:
!zip V2_intent_examples.zip V2*.json

  adding: V2_intent_examples_train1.json (deflated 82%)
  adding: V2_intent_examples_train2.json (deflated 83%)
  adding: V2_intent_examples_val_clean.json (deflated 81%)
  adding: V2_intent_examples_val.json (deflated 80%)


# III. Model BT1 (exp1)

## Initialize

In [ ]:
%env HOME=/content/drive/MyDrive
%cd ~/Research/ner/intent_classifier/model_env/dstc10_intent_BT1/
!ls -lrt

env: HOME=/content/drive/MyDrive
/content/drive/MyDrive/Research/ner/intent_classifier/model_env/dstc10_intent_BT1
total 428107
-rw------- 1 root root      6148 Aug 11 17:49 config.json
-rw------- 1 root root 438374583 Aug 11 17:49 pytorch_model.bin


In [ ]:
import torch

model_name = "bert-base-uncased"
batch_size = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!pip install transformers

  Using cached transformers-4.9.2-py3-none-any.whl (2.6 MB)
  Using cached huggingface_hub-0.0.12-py3-none-any.whl (37 kB)
  Using cached tokenizers-0.10.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Using cached sacremoses-0.0.45-py3-none-any.whl (895 kB)
  Using cached PyYAML-5.4.1-cp37-cp37m-manylinux1_x86_64.whl (636 kB)
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
#Loading Tokenizer, Config, and Model from drive
from transformers import AutoConfig, AutoTokenizer, AutoModel, AutoModelForSequenceClassification

model_name = "bert-base-uncased"

config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, padding = True, truncation = True)
#model = AutoModel.from_pretrained("./glue_model")
model = AutoModelForSequenceClassification.from_pretrained(".")

# Set device
model.to(device)

In [ ]:
#Checks for dict weight
state_dict = model.state_dict()
state_dict["bert.embeddings.word_embeddings.weight"]

tensor([[-0.0102, -0.0615, -0.0265,  ..., -0.0199, -0.0372, -0.0098],
        [-0.0117, -0.0600, -0.0323,  ..., -0.0168, -0.0401, -0.0107],
        [-0.0198, -0.0627, -0.0326,  ..., -0.0165, -0.0420, -0.0032],
        ...,
        [-0.0218, -0.0556, -0.0135,  ..., -0.0043, -0.0151, -0.0249],
        [-0.0462, -0.0565, -0.0019,  ...,  0.0157, -0.0139, -0.0095],
        [ 0.0015, -0.0821, -0.0160,  ..., -0.0081, -0.0475,  0.0753]],
       device='cuda:0')

## Simple Testing

In [ ]:
def _single_test(pair):
    result = tokenizer(pair, return_tensors="pt")
    result.to(device)
    model.eval()
    outputs = model(**result)
    #print("Output Dictionary Keys:\n", outputs.keys(),end='\n\n')
    #print("Sequence Logits:\n",outputs,end='\n\n')
    predictions = torch.topk(outputs.logits,3)
    #print('Preditions:\n',predictions,end='\n\n')
    #print(predictions)
    torch.cuda.empty_cache()
    return predictions

In [ ]:
#Try test sets here
sentence = "What is the length of the ride?"#"When can I check-in?"
pair = [sentence]
res = _single_test(pair)
#print(_single_test(pair))
int(res[1][0][0])


0

## Output New Label Field

In [ ]:
%cd ~/Research/ner/intent_classifier/model_updated_labels/

/content/drive/MyDrive/Research/ner/intent_classifier/model_updated_labels


### Method and initialize logs/updated_labels

In [ ]:
def _model_process_labels(labels, log, n):
    oops = 0
    for i in range(len(labels)):
        label = labels[i]
        pred = (int(_single_test([log[i][-1]['text']])))#or clean_text?
        if label['target'] == 1:
            for elem in label['knowledge']:
                elem['pred_cluster_id'] = pred
                if pred != elem['cluster_id']:
                    oops += 1
        else:
            label['pred_cluster_id'] = pred
            if pred!= 0:
                oops += 1

    out = open('model_'+str(n)+'.json', 'w')
    json.dump(labels, out, indent=4)
    out.close()
    print('# of Wrong predictions: ', oops)

In [ ]:
#Logs
import json

f1 = open('/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data/train/logs.json','r')
log_train = json.load(f1)
f1.close()

f2 = open('/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data/val/logs.json','r')
log_val = json.load(f2)
f2.close()

f3 = open('/content/drive/MyDrive/Research/ner/intent_classifier/exclude_template_logs.json','r')#if og then it would be cheat
log_eval = json.load(f3)
f3.close()

f4 = open('/content/drive/MyDrive/Research/ner/intent_classifier/DSTC10_clean_val_logs.json','r')
rlog_val = json.load(f4)
f4.close()

In [ ]:
#Labels
import json

dir = '../updated_labels/'

f1 = open(dir+'updated_DSTC9_task1_train_labels_phon_templates.json', 'r')
labels_train = json.load(f1)
f1.close()

f2 = open(dir+'updated_DSTC9_task1_val_labels_phon_templates.json', 'r')
labels_val = json.load(f2)
f2.close()

f3 = open(dir+'updated_exclude_labels.json', 'r')#if og then it would be cheat
labels_eval = json.load(f3)
f3.close()

f4 = open(dir+'/content/drive/MyDrive/Research/ner/intent_classifier/DSTC10_clean_val_labels.json', 'r')
rlabels_val = json.load(f4)
f4.close()


### Execute

In [ ]:
import json

label_lst = [labels_train, labels_val, labels_eval, rlabels_val]
log_lst = [log_train, log_val, log_eval, rlog_val]

for i in range(len(label_lst)):
    print('Now: ', i)
    _model_process_labels(label_lst[i], log_lst[i], i)




Now:  0
# of Wrong predictions:  452
Now:  1
# of Wrong predictions:  50
Now:  2
# of Wrong predictions:  61
Now:  3
# of Wrong predictions:  23


#Final

In [ ]:
%cd final_result

/content/drive/MyDrive/Research/ner/intent_classifier/final_result


In [ ]:
!cp /content/drive/MyDrive/Research/ner/text_classifier/final_results/kb_eval_final.json kb_eval_final.json

In [ ]:
import json

f1 = open('/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data/train/labels.json','r')
labels_train = json.load(f1)
f1.close()

f2 = open('/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data/val/labels.json','r')
labels_val = json.load(f2)
f2.close()

f3 = open('/content/drive/MyDrive/Research/ner/intent_classifier/exclude_labels.json','r')
labels_eval = json.load(f3)
f3.close()

f4 = open('/content/drive/MyDrive/Research/ner/intent_classifier/DSTC10_clean_val_labels.json','r')
rlabels_val = json.load(f4)
f4.close()

In [ ]:
#Logs
import json

f1 = open('/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data/train/logs.json','r')
log_train = json.load(f1)
f1.close()

f2 = open('/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data/val/logs.json','r')
log_val = json.load(f2)
f2.close()

f3 = open('/content/drive/MyDrive/Research/ner/intent_classifier/exclude_template_logs.json','r')#if og then it would be cheat
log_eval = json.load(f3)
f3.close()

f4 = open('/content/drive/MyDrive/Research/ner/intent_classifier/DSTC10_clean_val_logs.json','r')
rlog_val = json.load(f4)
f4.close()

In [ ]:
label_lst = [labels_train, labels_val, labels_eval, rlabels_val]
log_lst = [log_train, log_val, log_eval, rlog_val]
outnames = ['updated_DSTC9_train_labels.json',
            'updated_DSTC9_val_labels.json',
            'updated_DSTC9_excluded_eval_labels.json',
            'updated_DSTC10_clean_val_labels.json',
            
]
ff = open('kb_eval_final.json','r')
kb = json.load(ff)
ff.close()


for i in range(len(label_lst)):
    print('Processing ', i)
    label = label_lst[i]
    for j in range(len(label)):
        entry = label[j]
        if entry['target'] == True:
            domain = str(entry['knowledge'][0]['domain'])
            en_id = str(entry['knowledge'][0]['entity_id'])
            doc_id = str(entry['knowledge'][0]['doc_id'])
            print(domain,en_id,doc_id,sep = '\t')
            cluster_id = kb[domain][en_id]['docs'][doc_id]['intent_label']
            entry['knowledge'][0]['cluster_id'] = cluster_id
    f = open(outnames[i],'w')
    json.dump(label, f, indent=4)
    f.close()
            

In [ ]:
!zip updated_labels_wkb.zip *labels.json kb_eval_final.json

updating: kb_eval_final.json (deflated 92%)
updating: updated_DSTC10_clean_val_labels.json (deflated 93%)
updating: updated_DSTC9_excluded_eval_labels.json (deflated 90%)
updating: updated_DSTC9_train_labels.json (deflated 92%)
updating: updated_DSTC9_val_labels.json (deflated 92%)


# Others

In [ ]:
!ls -lrt
!pwd

total 38716
-rw------- 1 root root 13121130 Aug 31 22:18 kb_eval_final.json
-rw------- 1 root root  8636219 Aug 31 22:19 updated_DSTC9_train_labels.json
-rw------- 1 root root  1191914 Aug 31 22:19 updated_DSTC9_val_labels.json
-rw------- 1 root root   848616 Aug 31 22:19 updated_DSTC9_excluded_eval_labels.json
-rw------- 1 root root    76794 Aug 31 22:19 updated_DSTC10_clean_val_labels.json
-rw------- 1 root root  1883004 Aug 31 22:20 updated_labels_wkb.zip
-rw------- 1 root root    11998 Aug 31 22:22 intent_examples_val.json
-rw------- 1 root root  3020592 Aug 31 22:22 intent_examples_train1.json
-rw------- 1 root root    12071 Aug 31 22:23 intent_examples_val_clean.json
-rw------- 1 root root   672102 Aug 31 22:23 intent_examples.zip
-rw------- 1 root root    33326 Sep  1 00:39 V2_intent_examples_val.json
-rw------- 1 root root  6313393 Sep  1 00:39 V2_intent_examples_train1.json
-rw------- 1 root root  1331766 Sep  1 00:40 V2_intent_examples_train2.json
-rw------- 1 root root  1108